In [1]:
import qiskit
from qiskit.quantum_info import state_fidelity
import numpy as np
from scipy.sparse import linalg as LA
import qib
import matplotlib.pyplot as plt
import rqcopt as oc
import scipy
import scipy.sparse as sp


L = 8
J = (1, 1, 1)
h = (0, 0, 0)

latt_i = qib.lattice.IntegerLattice((2,), pbc=True)
field_i = qib.field.Field(qib.field.ParticleType.QUBIT, latt_i)
h_i = qib.HeisenbergHamiltonian(field_i, J, h).as_matrix()
H_i = sp.csr_matrix((2**L, 2**L))

# Expand
for i in range(L//2):
    H_ii = sp.eye(1)
    for j in range(i):
        H_ii = sp.kron(H_ii, sp.eye(2**2), format='csr')
    H_ii = sp.kron(H_ii, h_i, format='csr')
    for j in range(i+1, L//2):
        H_ii = sp.kron(H_ii, sp.eye(2**2), format='csr')
    H_i = H_i + H_ii

eigenvalues, eigenvectors = LA.eigsh(H_i, k=10)
idx = eigenvalues.argsort()
eigenvectors_sort = eigenvectors[:,idx]
init_state = eigenvectors_sort[:, 0]

# construct Hamiltonian
latt = qib.lattice.IntegerLattice((L, ), pbc=True)
field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)

hamil = qib.HeisenbergHamiltonian(field, J, h).as_matrix()
eigenvalues, eigenvectors = LA.eigsh(hamil, k=10)
idx = eigenvalues.argsort()
eigenvalues_sort = eigenvalues[idx]
eigenvectors_sort = eigenvectors[:,idx]
ground_state = eigenvectors_sort[:, 0]

print("Ground State Energy", eigenvalues_sort[0].real)
print("First Excited State Energy", eigenvalues_sort[4].real)
print("Average Energy: ", eigenvalues_sort[0]/L)
E_exact = eigenvalues_sort[0]/L

Ground State Energy -14.604373635748773
First Excited State Energy -10.798512593101359
Average Energy:  -1.8255467044685967


In [2]:
import sys
sys.path.append("../../src/adiabatic_gsp")
from adiabatic_gsp import run_adiabatic
from utils_gsp import construct_heisenberg_local_term


hloc = construct_heisenberg_local_term(J, h, 1)
qc_AQC = run_adiabatic(L, 2, 4, hloc, init_state, return_state=True, eigenvectors_sort=eigenvectors_sort)

Resulting Ground State Fidelity:  0.8580497308366762


In [3]:
"""
    Compressed-Controlled Time Evolution Operator that we optimized previously.
"""
import h5py
import sys
sys.path.append("../../src/brickwall_ansatz")
from utils import construct_heisenberg_local_term, construct_ising_local_term, get_params, reduce_list
from ansatz import ansatz, construct_ccU


# Best Performing Setting I observed so far was (2, 5).
L = 4
t = 0.25
eta, gamma = (2, 5)
nlayers = eta*gamma+gamma+1
perms = [[i for i in range(L)] if i%2==0 else [i for i in range(1, L)]+[0] for i in range(nlayers)]
Vs = []
with h5py.File(f"./results/heisenberg1d_000_L{L}_t{t}_layers{nlayers}_gamma{gamma}_eta{eta}.hdf5", "r") as f:
    Vs = f["Vlist"][:]

L = 8
latt = qib.lattice.IntegerLattice((L,), pbc=True)
field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)
hamil = qib.HeisenbergHamiltonian(field, J, h).as_matrix().toarray()
perms = [[i for i in range(L)] if i%2==0 else [i for i in range(1, L)]+[0] for i in range(nlayers)]

U = scipy.linalg.expm(-1j*hamil*t)
U_back = scipy.linalg.expm(1j*hamil*t)

print("Trotter error for same time, larger system: ", (np.linalg.norm(ansatz(reduce_list(Vs, gamma, eta), L, 
            reduce_list(perms, gamma, eta)) - U, ord=2)+np.linalg.norm(
        ansatz(Vs, L, perms) - U_back, ord=2))/2)

#  Controlled Two qubit Gates Decomposed & Optimized Previously.
Xlists_opt = {}
perms_qc = [[0, 1], [0, 2], [1, 2], [0, 2], [0, 1], [1, 2], [0, 2], [0, 1], [1, 2]]
for i in range(0, nlayers, eta+1):
    with h5py.File(f"./results/heisenberg1d_000_L4_t{t}_layers{nlayers}_gamma{gamma}_eta{eta}_CUs_APPROXS_n{len(perms_qc)}_layer{i}.hdf5", "r") as file:
        Xlists_opt[i] = file[f"Xlist_{i}"][:]

qc_cU = construct_ccU(L, eta, Vs, Xlists_opt, perms, perms_qc)


Trotter error for same time, larger system:  0.028758914127256757


In [4]:
"""
    Trotter error of the optimal gates construct.
"""

from qiskit import Aer, execute, transpile
from qiskit.converters import circuit_to_dag
from qiskit.providers.aer.noise import NoiseModel, errors

I2 = np.eye(2)
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])
ket_0 = np.array([[1],[0]])
ket_1 = np.array([[0],[1]])
rho_0_anc = ket_0 @ ket_0.T
rho_1_anc = ket_1 @ ket_1.T

U = scipy.linalg.expm(-1j * t * hamil)
cU = np.kron(rho_0_anc, U.conj().T) + np.kron(rho_1_anc, U)
backend = Aer.get_backend("unitary_simulator")
qc_unit = execute(transpile(qc_cU), backend).result().get_unitary(qc_cU, L+1).data
noise_model = NoiseModel()
dag = circuit_to_dag(transpile(qc_cU, basis_gates=noise_model.basis_gates+['unitary', 'initialize', 'cx']))
count_ops = dag.count_ops()

print(f"t={t}, Gate Count: ", count_ops['unitary'], " Trotter error: ", np.linalg.norm(qc_unit - cU, ord=2))

/tmp/ipykernel_32346/4053000614.py:21: DeprecationWarning: The function ``qiskit.execute_function.execute()`` is deprecated as of qiskit 0.46.0. It will be removed in the Qiskit 1.0 release. This function combines ``transpile`` and ``backend.run``, which is covered by ``Sampler`` :mod:`~qiskit.primitives`. Alternatively, you can also run :func:`.transpile` followed by ``backend.run()``.
  qc_unit = execute(transpile(qc_cU), backend).result().get_unitary(qc_cU, L+1).data


t=0.25, Gate Count:  256  Trotter error:  0.04239424550797945


In [5]:
"""

    Adiabatic + Iterative QPE

"""
import sys
sys.path.append("../../src/qpe")
from qpe import estimate_phases

def norm_mod2pi(theta):
    return np.pi - np.abs((theta%(2*np.pi)) - np.pi) 

In [18]:
def run_QPE(qc_prepared_state, qc_cU, basis_time, init_guess, Ns, final_digit, depolarizing_error):
    theta_prev = -init_guess
    est_prev = init_guess

    x1_error = errors.depolarizing_error(depolarizing_error*0.01, 1)
    x2_error = errors.depolarizing_error(depolarizing_error, 2)
    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(x1_error, ['u1', 'u2', 'u3'])
    noise_model.add_all_qubit_quantum_error(x2_error, ['cu3', 'cx', 'str', 'cy', 'cz'])
    
    Es = []
    ests_ = []
    thetas_ = []
    cxss_A = []
    mid_cbits = 0
    mid_errs_A = []
    for j in list(range(-1, final_digit+1, 1)):
        T = 2**j
    
        """qc_cU = qiskit.QuantumCircuit(L+1)
        U = scipy.linalg.expm(-1j * T * hamil)
        cU = np.kron(rho_0_anc, np.eye(2**L)) + np.kron(rho_1_anc, U)
        qc_cU.unitary(cU, [i for i in range(L+1)])"""
        theta_prev_ = theta_prev
        counts_real, counts_imag, cxs, qasm = estimate_phases(
                                                L, qc_C, eigenvalues_sort, T/2, basis_time,
                                                Ns, depolarizing_error, qc_cU, noise_model=noise_model,
                                                return_counts=True,
                                                get_cx=True, qasm=True
                                           )[0]
        cxss_A.append(cxs['unitary'])
        print('CXs: ', cxss_A[-1])
    
        phase_est_real = ((counts_real['0'] if '0' in counts_real else 0) - (counts_real['1'] if '1' in counts_real else 0)) /\
                    ((counts_real['0'] if '0' in counts_real else 0) + (counts_real['1'] if '1' in counts_real else 0))     
        phase_est_imag = ((counts_imag['0'] if '0' in counts_imag else 0) - (counts_imag['1'] if '1' in counts_imag else 0)) /\
                    ((counts_imag['0'] if '0' in counts_imag else 0) + (counts_imag['1'] if '1' in counts_imag else 0))
        phase = phase_est_real + 1j*phase_est_imag
                
        print("Exact Phase: ", np.exp(-1j * T * eigenvalues_sort[0]))
        print("Estimated Phase: ", phase)
                
        if phase.real != 0:
            angle = np.arctan(phase.imag/phase.real) if phase.real>0 else (np.pi + np.arctan(phase.imag/phase.real) if phase.imag>0 else \
                                                                               np.arctan(phase.imag/phase.real) - np.pi)
        else:
            angle = np.pi/2 if phase.imag > 0 else -np.pi/2

    
        if T>1:
            thetas = [(angle + k*2*np.pi)/T for k in range(T+1)]
            norms = np.array([norm_mod2pi(theta - theta_prev) for theta in thetas])
            id_ = np.argmin(norms)
                    
            ests = [-thetas[id_] - coe*np.pi for coe in range(2, 40, 2)]
            est = ests[0]
            for est_ in ests[1:]:
                est = est_ if np.abs(est_-est_prev) < np.abs(est-est_prev) else est
            ests_.append(est)
            thetas_.append(thetas[id_])
    
            #thetas_.sort()
            #theta_prev = thetas_[0]
            theta_prev = thetas_[-1]
        else:
            thetas = [(angle + k*np.pi)/T for k in range(0, 40, 2)]
            ests = [-theta for theta in thetas]
            est = ests[0]
            for est_ in ests[1:]:
                est = est_ if np.abs(est_-est_prev) < np.abs(est-est_prev) else est
            ests_.append(est)
            thetas_.append(-est)
            #print("Estimation Result: ", est)
            #print('Abs Error: ', np.abs(est-eigenvalues_sort[0]))
            #thetas_.sort()
            #theta_prev = thetas_[0]
            theta_prev = thetas_[-1]
    
        #ests_.sort()
        #est_prev = ests_[0]
        est_prev = ests_[-1]
        print("Final Estimation Result, Abs Error: ", np.abs(est_prev-eigenvalues_sort[0]))    
        print("Final Estimation Result, Relative Error: ", np.abs(est_prev-eigenvalues_sort[0])/np.abs(eigenvalues_sort[0]), '\n')    
        mid_errs_A.append(np.abs(est_prev-eigenvalues_sort[0]))
        Es.append(ests_)

In [24]:
# We can achieve up to relative error of 10^{-3} precision. Almost at the chemical accuracy!


qc_A = run_adiabatic(L, 2, 4, hloc, init_state, return_state=True, eigenvectors_sort=eigenvectors_sort)
qc_C = qiskit.QuantumCircuit(L+1, 1)
for q in range(L//2):
    qc_C.x(2*q)
    qc_C.h(2*q)
    qc_C.x(2*q+1)
    qc_C.cx(2*q, 2*q+1)
qc_C.append(qc_A.to_gate(), [i for i in range(L)])
qc_C.barrier()

run_QPE(qc_C, qc_cU, t, -20, 300, 2, 1e-3)

Resulting Ground State Fidelity:  0.8580497308366762
t:  0.25
nsteps:  1
getting counts
CXs:  352
Exact Phase:  (0.5242165109641607+0.8515850219634925j)
Estimated Phase:  (0.49333333333333335+0.6466666666666666j)
Final Estimation Result, Abs Error:  0.19980566088620222
Final Estimation Result, Relative Error:  0.01368122083627848 

t:  0.5
nsteps:  2
getting counts
CXs:  608
Exact Phase:  (-0.4503940992651241+0.8928298580060803j)
Estimated Phase:  (-0.34+0.6j)
Final Estimation Result, Abs Error:  0.04834231286427482
Final Estimation Result, Relative Error:  0.0033101257246625 

t:  1.0
nsteps:  4
getting counts
CXs:  1120
Exact Phase:  (-0.5942903106943151-0.8042505993873144j)
Estimated Phase:  (-0.62-0.5733333333333334j)
Final Estimation Result, Abs Error:  0.0940507272142046
Final Estimation Result, Relative Error:  0.006439901467871653 

t:  2.0
nsteps:  8
getting counts
CXs:  2144
Exact Phase:  (-0.2936380532297088+0.9559166771719524j)
Estimated Phase:  (-0.1+0.6533333333333333j)
F

In [23]:
from utils import cU_trotter
from qiskit.circuit.library import CYGate, CZGate, IGate

hloc1 = construct_heisenberg_local_term((J[0], 0, 0), h)
hloc2 = construct_heisenberg_local_term((0, J[1], 0), h)
hloc3 = construct_heisenberg_local_term((0, 0, J[2]), h)
hlocs = ((hloc1, hloc2, ), (hloc3, )) # H1 and H2
cgates = (CZGate, CYGate)

qc_cU_trotter = cU_trotter(t, L, hlocs, cgates, trotter_step=0.1, trotter_degree=2)
run_QPE(qc_C, qc_cU_trotter, t, -20, 300, 2, 1e-3)

t:  0.25
nsteps:  1
getting counts
CXs:  216
Exact Phase:  (0.5242165109641607+0.8515850219634925j)
Estimated Phase:  (0.32666666666666666+0.46j)
Final Estimation Result, Abs Error:  0.13141484809395543
Final Estimation Result, Relative Error:  0.008998321418747906 

t:  0.5
nsteps:  2
getting counts
CXs:  336
Exact Phase:  (-0.4503940992651241+0.8928298580060803j)
Estimated Phase:  (-0.12666666666666668+0.38666666666666666j)
Final Estimation Result, Abs Error:  0.15063744098230103
Final Estimation Result, Relative Error:  0.01031454307725795 

t:  1.0
nsteps:  4
getting counts
CXs:  576
Exact Phase:  (-0.5942903106943151-0.8042505993873144j)
Estimated Phase:  (-0.3466666666666667-0.24666666666666667j)
Final Estimation Result, Abs Error:  0.15799269740844402
Final Estimation Result, Relative Error:  0.010818176893373055 

t:  2.0
nsteps:  8
getting counts
CXs:  1056
Exact Phase:  (-0.2936380532297088+0.9559166771719524j)
Estimated Phase:  (-0.02+0.17333333333333334j)
Final Estimation R